# Graph traversal
https://mitre-attack.github.io/attack-navigator//#layerURL=https%3A%2F%2Fattack.mitre.org%2Fgroups%2FG0018%2FG0018-enterprise-layer.json

In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from pprint import pprint
from collections import Counter
from pyattck import Attck
import networkx as nx
from pyvis.network import Network
from itertools import product
import itertools
import matplotlib as plt

# Init pyatt lib
attck = Attck()

In [2]:
tacticcolumns = []
for tactic in attck.enterprise.tactics:
    templist = []
    for teachniques in tactic.techniques:
        if 'Windows' in teachniques.platforms:
            templist.append(teachniques.id)
    tacticcolumns.append((tactic.id, sorted(templist)))

In [3]:
# Gather and store edge pairs using n-gram(Works)
edgelist = []

for actor in attck.enterprise.actors:
    tempactorlist = []
    edge_i = [x.id for x in actor.techniques]
    
    for items in tacticcolumns:
        temptech = []
        for tech in edge_i:
            if tech in items[1]:
                temptech.append(tech)
        if temptech:
            tempactorlist.append(temptech)
    edgelist.append(tempactorlist)
    
edgelist[0]

[['T1571', 'T1105'],
 ['T1027', 'T1036.004'],
 ['T1053.005', 'T1204.002', 'T1059.005'],
 ['T1566.001'],
 ['T1053.005'],
 ['T1053.005']]

In [4]:
newedgelist = []
for x in edgelist:
    newedgelist.append(list(itertools.combinations(itertools.chain(*x), 2)))
newedgelist = [y for x in newedgelist for y in x]

In [5]:
# newgraph = nx.DiGraph(newedgelist[100:250])
# nt = Network('1000px', '1000px', directed=True)
# nt.toggle_physics(False)
# nt.from_nx(newgraph)
# nt.show('nx.html')

In [6]:
# User counter to count number of time edges appear to create weights
counter = Counter(sorted(newedgelist))
counter

Counter({('T1001', 'T1001.002'): 1,
         ('T1001', 'T1003'): 1,
         ('T1001', 'T1003.001'): 1,
         ('T1001', 'T1003.006'): 1,
         ('T1001', 'T1007'): 1,
         ('T1001', 'T1012'): 1,
         ('T1001', 'T1016'): 1,
         ('T1001', 'T1018'): 1,
         ('T1001', 'T1021.001'): 1,
         ('T1001', 'T1021.002'): 1,
         ('T1001', 'T1027'): 1,
         ('T1001', 'T1027.005'): 1,
         ('T1001', 'T1033'): 1,
         ('T1001', 'T1041'): 1,
         ('T1001', 'T1046'): 1,
         ('T1001', 'T1047'): 1,
         ('T1001', 'T1049'): 1,
         ('T1001', 'T1053.005'): 3,
         ('T1001', 'T1055'): 2,
         ('T1001', 'T1056.001'): 1,
         ('T1001', 'T1057'): 1,
         ('T1001', 'T1059.001'): 1,
         ('T1001', 'T1059.003'): 1,
         ('T1001', 'T1059.005'): 1,
         ('T1001', 'T1059.006'): 1,
         ('T1001', 'T1069.001'): 1,
         ('T1001', 'T1070.001'): 1,
         ('T1001', 'T1070.004'): 1,
         ('T1001', 'T1078'): 4,
         ('T

In [8]:
# Store each value in array list
sourcelist = []
targetlist = []
weightlist = []
for key, value in counter.items():
    sourcelist.append(key[0])
    targetlist.append(key[1])
    weightlist.append(value)
    
# check if the len are same value
print(len(sourcelist))
print(len(targetlist))
print(len(weightlist))

17543
17543
17543


In [9]:
# Store the nodes and weights in dataframe
pd.set_option('display.max_rows', 20)
edges = pd.DataFrame(
    {
        "source": sourcelist,#sourceId,
        "target": targetlist, #targetlist
        "weight": weightlist
    }
)
edges

,source,target,weight
0,T1001,T1001.002,1
1,T1001,T1003,1
2,T1001,T1003.001,1
3,T1001,T1003.006,1
4,T1001,T1007,1
...,...,...,...
17538,T1606.002,T1562.002,2
17539,T1606.002,T1562.004,2
17540,T1606.002,T1566.001,1
17541,T1606.002,T1566.002,1


In [10]:
# Create a graph using the dataframe
G = nx.from_pandas_edgelist(edges, edge_attr=True,create_using=nx.DiGraph())

In [11]:
dfstree = nx.dfs_tree(G, "T1030")
list(dfstree.edges())

[('T1030', 'T1021.006'),
 ('T1021.006', 'T1021.002'),
 ('T1021.002', 'T1021.001'),
 ('T1021.001', 'T1021.005'),
 ('T1021.005', 'T1053.005'),
 ('T1053.005', 'T1020'),
 ('T1020', 'T1052.001'),
 ('T1052.001', 'T1055.001'),
 ('T1055.001', 'T1007'),
 ('T1007', 'T1012'),
 ('T1012', 'T1016'),
 ('T1016', 'T1016.001'),
 ('T1016.001', 'T1018'),
 ('T1018', 'T1033'),
 ('T1033', 'T1010'),
 ('T1010', 'T1041'),
 ('T1041', 'T1029'),
 ('T1029', 'T1547.001'),
 ('T1547.001', 'T1037'),
 ('T1037', 'T1055.002'),
 ('T1055.002', 'T1014'),
 ('T1014', 'T1027'),
 ('T1027', 'T1027.001'),
 ('T1027.001', 'T1027.002'),
 ('T1027.002', 'T1027.004'),
 ('T1027.004', 'T1027.003'),
 ('T1027.003', 'T1036.005'),
 ('T1036.005', 'T1027.005'),
 ('T1027.005', 'T1046'),
 ('T1046', 'T1037.001'),
 ('T1037.001', 'T1055'),
 ('T1055', 'T1036'),
 ('T1036', 'T1036.003'),
 ('T1036.003', 'T1047'),
 ('T1047', 'T1048.002'),
 ('T1048.002', 'T1078'),
 ('T1078', 'T1036.004'),
 ('T1036.004', 'T1040'),
 ('T1040', 'T1003'),
 ('T1003', 'T1003.001

In [23]:
def depthfirstsearch(graph, node):
    visited = set() # Set to keep track of visited nodes.
    result = set()

    def dfs(visited, graph, node):
        if node not in visited:
            ttpdetect = ttpsearch(node)
            if ttpdetect:
                result.add(node)
            visited.add(node)
            weightlist = [G[node][x]["weight"] for x in G.neighbors(node) if x not in visited]
            maxweight = max(weightlist, default=0)
            maxweight_node = [x for x in G.neighbors(node) if G[node][x]["weight"] == maxweight]
            
#             print("current node:", node)
#             print("neighbours:", list(G.neighbors(node)))
#             print("weightlist:", weightlist)
#             print("max weight calculated:", maxweight)
#             print("nodes found from max weight:", maxweight_node)

            for node in maxweight_node:
                dfs(visited, graph, node)
    dfs(visited, graph, node)
    return result

# emulated ttpsearch using random, with true propability = 0.1
def ttpsearch(ttpid):
    return random.randrange(100) < 10

In [29]:
# by id
print(depthfirstsearch(edges, "T1030"))

{'T1197', 'T1133', 'T1091', 'T1534', 'T1036.005', 'T1546.010', 'T1199', 'T1046', 'T1499', 'T1049', 'T1014', 'T1053.002', 'T1068', 'T1484.001', 'T1059.001', 'T1547.001', 'T1029', 'T1140', 'T1082'}
